# Module 6: Precipitation Frequency Analysis with NOAA Atlas 14
## Design Storm Generation for Multiple AEP Events ☔🌧️

### Welcome to Design Storm Analysis!
This module focuses on generating design storms for hydrologic and hydraulic modeling using NOAA Atlas 14 precipitation frequency data. You'll learn to automate the creation of multiple design storm hyetographs and generate professional summary tables for engineering reports.

### What You'll Accomplish Today:
✅ Access NOAA Atlas 14 precipitation frequency data via API  
✅ Upload and apply custom temporal distributions (MSE 6 24-hour)  
✅ Generate design storms for multiple AEP events  
✅ Create professional hyetographs with proper formatting  
✅ Build summary tables for engineering reports  
✅ Export data for HEC-HMS and HEC-RAS models  
✅ Batch process multiple return periods efficiently  

### Module Structure:
1. **Mental Models** - Design storm concepts for H&H
2. **NOAA Atlas 14** - Accessing precipitation frequency data
3. **Temporal Distributions** - MSE 6 24-hour pattern
4. **Design Storm Generation** - Creating hyetographs
5. **Batch Processing** - Multiple AEP events
6. **Summary Tables** - Professional reporting
7. **Model Export** - HEC-HMS/RAS formats
8. **Complete Workflow** - Production-ready automation

---

## Part 1: Mental Models - Design Storms for H&H 🧠

### What is a Design Storm?

A **design storm** is a synthetic rainfall event used for hydrologic/hydraulic modeling:
- Based on **statistical analysis** of historical data
- Represents a specific **return period** or **probability**
- Used for **regulatory compliance** (FEMA, local standards)

### Key Terminology

| Term | Definition | Example |
|------|------------|----------|
| **ARI** | Annual Recurrence Interval | 100-year storm |
| **AEP** | Annual Exceedance Probability | 1% (= 100-year) |
| **Duration** | Total storm length | 24 hours |
| **Depth** | Total precipitation | 6.5 inches |
| **Hyetograph** | Rainfall vs time graph | Design storm time series |
| **Temporal Distribution** | How rainfall is spread over time | MSE 6, SCS Type II |

### AEP ↔ ARI Conversion

```
AEP (%) = 100 / ARI (years)
ARI (years) = 100 / AEP (%)
```

| AEP | ARI | Common Use |
|-----|-----|------------|
| 50% | 2-year | Frequent event |
| 10% | 10-year | Minor flooding |
| 4% | 25-year | Moderate flooding |
| 2% | 50-year | Major flooding |
| 1% | 100-year | **Base flood (FEMA)** |
| 0.5% | 200-year | Extreme event |
| 0.2% | 500-year | Critical infrastructure |

### The Design Storm Workflow

```
1. NOAA Atlas 14    →  Get total precipitation depth for AEP/Duration
2. Temporal Pattern →  Apply distribution (MSE 6, SCS Type II, etc.)
3. Hyetograph      →  Create time series at desired interval
4. Model Input     →  Export to HEC-HMS/RAS format
```

### What Makes This Module Different

Instead of creating **one storm at a time**, you'll learn to:
- Generate **multiple AEP events** in one run
- Create **comparative tables** for reports
- **Batch export** for model input
- Build **reusable workflows** for any project

## Part 2: Setting Up Your Workspace 🛠️

### Required Libraries

We'll use:
- `pandas` - Time series and data manipulation
- `numpy` - Numerical calculations
- `matplotlib` - Hyetograph plotting
- `urllib` - NOAA Atlas 14 API access (built-in)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import urllib.request
import urllib.error
import ast
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)
pd.set_option('display.precision', 2)

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully!")
print("   Ready for precipitation frequency analysis")

## Part 3: NOAA Atlas 14 API Access 🌐

### What is NOAA Atlas 14?

**NOAA Atlas 14** is the authoritative source for precipitation frequency estimates in the United States:
- Published by NOAA's Hydrometeorological Design Studies Center (HDSC)
- Based on statistical analysis of 50+ years of rainfall data
- Covers all of CONUS (with regional volumes)
- Updated periodically with new data

### API Access Strategy

We'll implement the API function based on production code from RAS Commander:
1. Use `urllib` instead of `requests` for better compatibility
2. Parse JavaScript-style response from NOAA
3. Handle network errors gracefully
4. Provide fallback for manual data entry

In [ ]:
def get_atlas14_data(latitude, longitude, units='english', timeout=30):
    """
    Download NOAA Atlas 14 precipitation frequency data for a location.
    
    Based on RAS Commander implementation for production reliability.
    
    Parameters:
    -----------
    latitude : float
        Latitude in decimal degrees (positive for Northern Hemisphere)
    longitude : float
        Longitude in decimal degrees (negative for Western Hemisphere)
    units : str
        'english' for inches or 'metric' for millimeters
    timeout : int
        Request timeout in seconds
    
    Returns:
    --------
    pd.DataFrame : Precipitation depths (inches or mm)
        Rows: Durations (5-min through 60-day)
        Columns: Return periods (1-yr through 1000-yr)
    
    Raises:
    -------
    ConnectionError : If unable to reach NOAA API
    ValueError : If location is outside Atlas 14 coverage
    """
    
    # NOAA HDSC API endpoint
    base_url = "https://hdsc.nws.noaa.gov/cgi-bin/hdsc/new/cgi_readH5.py"
    
    # Build request parameters
    params = {
        'lat': latitude,
        'lon': longitude,
        'type': 'pf',  # Precipitation frequency
        'data': 'depth',
        'units': units,
        'series': 'pds'  # Partial duration series
    }
    
    # Build URL with query string
    query_string = '&'.join(f"{k}={v}" for k, v in params.items())
    url = f"{base_url}?{query_string}"
    
    print(f"📡 Requesting Atlas 14 data for ({latitude:.4f}, {longitude:.4f})...")
    print(f"   Units: {units}")
    
    try:
        # Create request with proper headers
        request = urllib.request.Request(url)
        request.add_header('User-Agent', 'Python-urllib/3.0 (Educational)')
        
        # Make request
        with urllib.request.urlopen(request, timeout=timeout) as response:
            content = response.read().decode('utf-8')
        
        # Parse the response (JavaScript-style variable assignments)
        data_dict = {}
        for line in content.split('\n'):
            line = line.strip()
            if '=' in line and not line.startswith('#'):
                try:
                    # Split on first '=' only
                    var_name, value_str = line.split('=', 1)
                    var_name = var_name.strip()
                    value_str = value_str.strip()
                    
                    # Remove trailing semicolon
                    if value_str.endswith(';'):
                        value_str = value_str[:-1].strip()
                    
                    # Parse value safely
                    try:
                        value = ast.literal_eval(value_str)
                    except (ValueError, SyntaxError):
                        value = value_str.strip('"\'')
                    
                    data_dict[var_name] = value
                except Exception:
                    continue
        
        # Extract quantiles array (precipitation depths)
        quantiles = data_dict.get('quantiles', [])
        
        if not quantiles:
            raise ValueError("No precipitation data in API response")
        
        # Standard durations and return periods
        duration_labels = ['5-min', '10-min', '15-min', '30-min', '60-min', 
                          '2-hr', '3-hr', '6-hr', '12-hr', '24-hr', 
                          '2-day', '3-day', '4-day', '7-day', '10-day', 
                          '20-day', '30-day', '45-day', '60-day']
        
        ari_labels = ['1-yr', '2-yr', '5-yr', '10-yr', '25-yr', 
                     '50-yr', '100-yr', '200-yr', '500-yr', '1000-yr']
        
        # Build DataFrame
        num_durations = min(len(quantiles), len(duration_labels))
        num_aris = min(len(quantiles[0]) if quantiles else 0, len(ari_labels))
        
        precip_data = {}
        for ari_idx in range(num_aris):
            values = []
            for dur_idx in range(num_durations):
                val = quantiles[dur_idx][ari_idx] if ari_idx < len(quantiles[dur_idx]) else np.nan
                # Convert to float if string
                if isinstance(val, str):
                    try:
                        val = float(val)
                    except (ValueError, TypeError):
                        val = np.nan
                values.append(val)
            precip_data[ari_labels[ari_idx]] = values
        
        df = pd.DataFrame(precip_data, index=duration_labels[:num_durations])
        
        region = data_dict.get('region', 'Unknown')
        print(f"✅ Downloaded Atlas 14 data successfully!")
        print(f"   Region: {region}")
        print(f"   Coverage: {num_durations} durations × {num_aris} return periods")
        
        return df
        
    except urllib.error.URLError as e:
        error_msg = str(e)
        print(f"\n❌ Connection error: {error_msg[:100]}")
        print("\n   This may be due to:")
        print("   • Network restrictions (firewall, proxy, VPN)")
        print("   • NOAA server temporarily unavailable")
        print("   • Location outside Atlas 14 coverage")
        print("\n   📝 Use manual data entry method (see next cell)")
        raise ConnectionError(f"Cannot access NOAA API: {e}")
    
    except Exception as e:
        print(f"\n❌ Error processing Atlas 14 data: {str(e)[:100]}")
        raise

print("✅ Atlas 14 API function defined!")
print("   Production-ready implementation based on RAS Commander")

### Get Atlas 14 Data for Your Location

**Try the API first:**

In [ ]:
# Enter your project coordinates
# Example: Minneapolis-St. Paul
PROJECT_LAT = 44.9778
PROJECT_LON = -93.2650
PROJECT_NAME = "Minneapolis-St. Paul"

# Try to download Atlas 14 data
try:
    atlas14_data = get_atlas14_data(PROJECT_LAT, PROJECT_LON, units='english')
    
    print(f"\n📊 Precipitation Frequency Data for {PROJECT_NAME}")
    print(f"   Location: ({PROJECT_LAT:.4f}°N, {PROJECT_LON:.4f}°W)\n")
    print(atlas14_data)
    
    print("\n🎯 Common Design Storm Depths (24-hour):")
    print(f"   10-year:   {atlas14_data.loc['24-hr', '10-yr']:.2f} inches")
    print(f"   25-year:   {atlas14_data.loc['24-hr', '25-yr']:.2f} inches")
    print(f"   50-year:   {atlas14_data.loc['24-hr', '50-yr']:.2f} inches")
    print(f"   100-year:  {atlas14_data.loc['24-hr', '100-yr']:.2f} inches")
    print(f"   500-year:  {atlas14_data.loc['24-hr', '500-yr']:.2f} inches")
    
    API_SUCCESS = True
    
except Exception as e:
    print(f"\n⚠️  API access failed")
    API_SUCCESS = False

### Manual Data Entry (If API Failed)

If the API couldn't connect, manually enter Atlas 14 data:

**How to get your data:**
1. Visit: https://hdsc.nws.noaa.gov/pfds/
2. Click on your project location
3. Select "Precipitation Frequency Data Server (PFDS)"
4. Copy the values from the table
5. Paste into the code below

In [ ]:
# Only run this cell if API failed above

if not API_SUCCESS:
    # Example: Minneapolis-St. Paul (NOAA Atlas 14 Volume 8)
    # Replace with YOUR location's values from NOAA website
    
    atlas14_data = pd.DataFrame({
        '1-yr':   [0.31, 0.42, 0.51, 0.72, 0.96, 1.34, 1.58, 2.07, 2.60, 3.11, 3.48, 3.70, 3.88, 4.24, 4.49, 5.10, 5.57, 6.29, 6.90],
        '2-yr':   [0.37, 0.51, 0.62, 0.88, 1.18, 1.65, 1.95, 2.58, 3.25, 3.93, 4.43, 4.73, 4.96, 5.45, 5.79, 6.59, 7.21, 8.15, 8.94],
        '5-yr':   [0.45, 0.62, 0.76, 1.09, 1.46, 2.07, 2.45, 3.28, 4.17, 5.09, 5.77, 6.18, 6.50, 7.17, 7.63, 8.71, 9.54, 10.81, 11.87],
        '10-yr':  [0.52, 0.71, 0.87, 1.25, 1.69, 2.40, 2.85, 3.86, 4.93, 6.06, 6.91, 7.42, 7.82, 8.66, 9.23, 10.56, 11.58, 13.13, 14.43],
        '25-yr':  [0.61, 0.84, 1.03, 1.48, 2.01, 2.88, 3.42, 4.68, 6.01, 7.45, 8.54, 9.19, 9.70, 10.79, 11.52, 13.20, 14.49, 16.45, 18.10],
        '50-yr':  [0.68, 0.94, 1.16, 1.67, 2.27, 3.26, 3.88, 5.34, 6.89, 8.57, 9.87, 10.64, 11.24, 12.54, 13.40, 15.39, 16.91, 19.22, 21.16],
        '100-yr': [0.76, 1.05, 1.29, 1.87, 2.54, 3.67, 4.38, 6.06, 7.85, 9.80, 11.32, 12.23, 12.93, 14.46, 15.48, 17.80, 19.56, 22.27, 24.52],
        '200-yr': [0.84, 1.16, 1.43, 2.07, 2.82, 4.09, 4.89, 6.81, 8.85, 11.09, 12.84, 13.89, 14.70, 16.48, 17.66, 20.34, 22.36, 25.49, 28.09],
        '500-yr': [0.95, 1.32, 1.63, 2.36, 3.22, 4.69, 5.61, 7.87, 10.27, 12.93, 15.00, 16.26, 17.23, 19.37, 20.78, 23.97, 26.37, 30.12, 33.22],
        '1000-yr':[1.05, 1.46, 1.80, 2.61, 3.57, 5.20, 6.23, 8.75, 11.46, 14.47, 16.82, 18.25, 19.36, 21.80, 23.41, 27.03, 29.74, 34.03, 37.56]
    }, index=['5-min', '10-min', '15-min', '30-min', '60-min', 
              '2-hr', '3-hr', '6-hr', '12-hr', '24-hr', '2-day', '3-day', 
              '4-day', '7-day', '10-day', '20-day', '30-day', '45-day', '60-day'])
    
    print(f"✅ Atlas 14 data loaded manually ({PROJECT_NAME})")
    print(f"\n📊 Precipitation Frequency Data:\n")
    print(atlas14_data)
    
    print("\n🎯 Common Design Storm Depths (24-hour):")
    print(f"   10-year:   {atlas14_data.loc['24-hr', '10-yr']:.2f} inches")
    print(f"   100-year:  {atlas14_data.loc['24-hr', '100-yr']:.2f} inches")
    print(f"   500-year:  {atlas14_data.loc['24-hr', '500-yr']:.2f} inches")
else:
    print("✅ Atlas 14 data already loaded from API")

## Part 4: Temporal Distribution - MSE 6 24-Hour Pattern 📊

### What is a Temporal Distribution?

A **temporal distribution** defines how total precipitation is spread over the storm duration:
- Atlas 14 gives you **total depth** (e.g., 6.5 inches)
- Temporal distribution tells you **when** it falls

### Common Distributions

- **SCS Type II**: Standard for most of US, peak at ~12 hours
- **SCS Type IA**: Pacific maritime climate
- **SCS Type III**: Gulf Coast and Florida
- **MSE 6**: Minnesota-specific 24-hour distribution

### Upload Your Distribution File

**Please upload** your `Rainfall_Depth_vs__Time_Table.csv` file using the file upload button below.

**Expected format:**
- Column 1: Time (hours from start)
- Column 2: Cumulative depth (as fraction of total, 0.0 to 1.0)

In [ ]:
# Upload temporal distribution file
from google.colab import files

print("📁 Please upload your temporal distribution CSV file")
print("   Expected: Rainfall_Depth_vs__Time_Table.csv")
print("   Format: [Time_hours, Cumulative_fraction]\n")

uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {filename}")

In [ ]:
# Load and process the temporal distribution
temporal_dist = pd.read_csv(filename)

# Display the distribution
print("📊 Temporal Distribution Data:")
print(f"   Shape: {temporal_dist.shape[0]} time steps\n")
print(temporal_dist.head(10))

# Get column names (first column = time, second = cumulative)
time_col = temporal_dist.columns[0]
cumul_col = temporal_dist.columns[1]

print(f"\n✅ Distribution loaded successfully!")
print(f"   Time column: '{time_col}'")
print(f"   Cumulative column: '{cumul_col}'")
print(f"   Duration: {temporal_dist[time_col].max():.1f} hours")

# Rename for easier use
temporal_dist = temporal_dist.rename(columns={
    time_col: 'time_hr',
    cumul_col: 'cumulative_fraction'
})

In [ ]:
# Visualize the temporal distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cumulative distribution
ax1.plot(temporal_dist['time_hr'], temporal_dist['cumulative_fraction'], 
         'o-', linewidth=2, markersize=4, color='darkblue')
ax1.fill_between(temporal_dist['time_hr'], temporal_dist['cumulative_fraction'], 
                 alpha=0.3, color='steelblue')
ax1.set_xlabel('Time (hours)', fontweight='bold')
ax1.set_ylabel('Cumulative Depth (fraction of total)', fontweight='bold')
ax1.set_title('MSE 6 Cumulative Distribution', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, temporal_dist['time_hr'].max())
ax1.set_ylim(0, 1.0)

# Calculate incremental (difference between successive cumulative values)
incremental = temporal_dist['cumulative_fraction'].diff().fillna(temporal_dist['cumulative_fraction'].iloc[0])

# Incremental distribution
ax2.bar(temporal_dist['time_hr'], incremental, 
       width=temporal_dist['time_hr'].diff().median(), 
       color='darkgreen', alpha=0.7, edgecolor='darkgreen')
ax2.set_xlabel('Time (hours)', fontweight='bold')
ax2.set_ylabel('Incremental Depth (fraction of total)', fontweight='bold')
ax2.set_title('MSE 6 Incremental Distribution', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, temporal_dist['time_hr'].max())

# Find peak
peak_idx = incremental.idxmax()
peak_time = temporal_dist.loc[peak_idx, 'time_hr']
ax2.annotate('Peak', xy=(peak_time, incremental.iloc[peak_idx]), 
            xytext=(peak_time+2, incremental.iloc[peak_idx]+0.05),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, fontweight='bold', color='red')

plt.tight_layout()
plt.show()

print(f"\n📈 Distribution characteristics:")
print(f"   Peak occurs at: {peak_time:.1f} hours ({peak_time/24*100:.0f}% of duration)")
print(f"   Peak increment: {incremental.iloc[peak_idx]:.3f} (fraction of total)")

## Part 5: Design Storm Hyetograph Generation 🌧️

### The Hyetograph Function

This function creates a design storm hyetograph by:
1. Taking total precipitation depth from Atlas 14
2. Applying the temporal distribution
3. Interpolating to desired time interval
4. Returning a complete time series

In [ ]:
def generate_design_storm(total_depth_in, temporal_dist, interval_min=6, start_datetime=None):
    """
    Generate design storm hyetograph from temporal distribution.
    
    Parameters:
    -----------
    total_depth_in : float
        Total precipitation depth (inches) from Atlas 14
    temporal_dist : pd.DataFrame
        Temporal distribution with columns ['time_hr', 'cumulative_fraction']
    interval_min : int
        Time interval for output (minutes)
    start_datetime : str or datetime, optional
        Start date/time for the storm (default: '2024-01-01 00:00')
    
    Returns:
    --------
    pd.DataFrame : Hyetograph with columns:
        - datetime: Timestamp
        - time_hr: Time from start (hours)
        - incremental_in: Precipitation per interval (inches)
        - cumulative_in: Cumulative precipitation (inches)
        - intensity_in_hr: Rainfall intensity (inches/hour)
    """
    
    # Default start time
    if start_datetime is None:
        start_datetime = pd.Timestamp('2024-01-01 00:00:00')
    else:
        start_datetime = pd.Timestamp(start_datetime)
    
    # Get storm duration from temporal distribution
    duration_hr = temporal_dist['time_hr'].max()
    
    # Create output time array at desired interval
    num_intervals = int(duration_hr * 60 / interval_min) + 1
    output_time_hr = np.linspace(0, duration_hr, num_intervals)
    
    # Interpolate cumulative distribution to output intervals
    cumulative_fraction = np.interp(output_time_hr, 
                                     temporal_dist['time_hr'], 
                                     temporal_dist['cumulative_fraction'])
    
    # Convert to actual depths
    cumulative_in = cumulative_fraction * total_depth_in
    
    # Calculate incremental depths
    incremental_in = np.diff(cumulative_in, prepend=0)
    
    # Calculate intensity (in/hr)
    intensity_in_hr = incremental_in / (interval_min / 60)
    
    # Create datetime index
    datetime_index = pd.date_range(start=start_datetime, 
                                   periods=num_intervals, 
                                   freq=f'{interval_min}min')
    
    # Build DataFrame
    hyetograph = pd.DataFrame({
        'datetime': datetime_index,
        'time_hr': output_time_hr,
        'incremental_in': incremental_in,
        'cumulative_in': cumulative_in,
        'intensity_in_hr': intensity_in_hr
    })
    
    return hyetograph

print("✅ Design storm generator defined!")
print("   Ready to create hyetographs from Atlas 14 data")

### Example: Generate Single Design Storm

Let's generate one storm to verify the function works:

In [ ]:
# Generate 100-year, 24-hour design storm
total_depth_100yr = atlas14_data.loc['24-hr', '100-yr']

storm_100yr = generate_design_storm(
    total_depth_in=total_depth_100yr,
    temporal_dist=temporal_dist,
    interval_min=6
)

print(f"✅ Generated 100-year, 24-hour design storm")
print(f"   Total depth: {total_depth_100yr:.2f} inches")
print(f"   Intervals: {len(storm_100yr)}")
print(f"   Time step: 15 minutes")
print(f"   Peak intensity: {storm_100yr['intensity_in_hr'].max():.2f} in/hr")
print(f"\n📊 Storm hyetograph (first 10 intervals):\n")
print(storm_100yr[['datetime', 'incremental_in', 'intensity_in_hr']].head(10))

In [ ]:
# Plot the example storm
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Incremental precipitation
ax1.bar(storm_100yr['time_hr'], storm_100yr['incremental_in'], 
       width=0.2, color='darkblue', alpha=0.7)
ax1.set_ylabel('Incremental Precipitation (inches)', fontweight='bold')
ax1.set_title(f'100-Year, 24-Hour Design Storm - {PROJECT_NAME}\nTotal: {total_depth_100yr:.2f} inches', 
             fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 24)

# Mark peak
peak_idx = storm_100yr['incremental_in'].idxmax()
peak_time = storm_100yr.loc[peak_idx, 'time_hr']
peak_value = storm_100yr.loc[peak_idx, 'incremental_in']
ax1.annotate(f'Peak: {storm_100yr.loc[peak_idx, "intensity_in_hr"]:.2f} in/hr', 
            xy=(peak_time, peak_value),
            xytext=(peak_time+2, peak_value+0.1),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, fontweight='bold', color='red')

# Cumulative precipitation
ax2.plot(storm_100yr['time_hr'], storm_100yr['cumulative_in'], 
        color='darkblue', linewidth=2.5)
ax2.fill_between(storm_100yr['time_hr'], storm_100yr['cumulative_in'], 
                alpha=0.3, color='steelblue')
ax2.set_xlabel('Time (hours)', fontweight='bold', fontsize=11)
ax2.set_ylabel('Cumulative Precipitation (inches)', fontweight='bold')
ax2.set_title('Cumulative Rainfall Curve', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 24)

plt.tight_layout()
plt.show()

## Part 6: Batch Processing Multiple AEP Events 🔄

### The Workflow

Real H&H projects require multiple design storms:
- **Flood frequency analysis**: 10-yr, 50-yr, 100-yr, 500-yr
- **Risk assessment**: Range of probabilities
- **Design optimization**: Compare alternatives

### Batch Generator Function

In [ ]:
def generate_aep_suite(atlas14_data, temporal_dist, aep_list, duration='24-hr', 
                       interval_min=6, output_dir=None):
    """
    Generate design storms for multiple AEP events.
    
    Parameters:
    -----------
    atlas14_data : pd.DataFrame
        Atlas 14 precipitation frequency data
    temporal_dist : pd.DataFrame
        Temporal distribution pattern
    aep_list : list of int
        AEP values (return periods in years), e.g., [10, 25, 50, 100, 500]
    duration : str
        Storm duration (must match Atlas 14 index), e.g., '24-hr'
    interval_min : int
        Output time interval (minutes)
    output_dir : str or Path, optional
        Directory to save CSV files (created if doesn't exist)
    
    Returns:
    --------
    dict : Dictionary with AEP as keys, hyetographs as values
    pd.DataFrame : Summary table of storm characteristics
    """
    
    print(f"🔄 Generating design storm suite...")
    print(f"   Duration: {duration}")
    print(f"   AEP Events: {aep_list}")
    print(f"   Time interval: {interval_min} minutes\n")
    
    storms = {}
    summary_data = []
    
    # Create output directory if specified
    if output_dir is not None:
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"📁 Output directory: {output_dir}\n")
    
    for aep in aep_list:
        print(f"   Generating {aep}-year storm...")
        
        # Get total depth
        col_name = f'{aep}-yr'
        total_depth = atlas14_data.loc[duration, col_name]
        
        # Generate hyetograph
        hyeto = generate_design_storm(
            total_depth_in=total_depth,
            temporal_dist=temporal_dist,
            interval_min=interval_min
        )
        
        storms[aep] = hyeto
        
        # Calculate statistics
        summary_data.append({
            'AEP': f'{aep}-year',
            'Probability': f'{100/aep:.2f}%',
            'Total_Depth_in': total_depth,
            'Peak_Intensity_in_hr': hyeto['intensity_in_hr'].max(),
            'Peak_Time_hr': hyeto.loc[hyeto['intensity_in_hr'].idxmax(), 'time_hr'],
            'Intervals': len(hyeto)
        })
        
        # Save to CSV if output directory specified
        if output_dir is not None:
            filename = output_dir / f'Storm_{aep}yr_{duration}.csv'
            hyeto.to_csv(filename, index=False)
            print(f"      Saved: {filename.name}")
    
    # Create summary table
    summary = pd.DataFrame(summary_data)
    
    print(f"\n✅ Generated {len(aep_list)} design storms successfully!\n")
    
    return storms, summary

print("✅ Batch processing function defined!")

### Generate Your Design Storm Suite

Customize the AEP list for your project:

In [ ]:
# Define your AEP events
AEP_EVENTS = [10, 25, 50, 100, 500]  # Modify as needed

# Generate all storms
design_storms, storm_summary = generate_aep_suite(
    atlas14_data=atlas14_data,
    temporal_dist=temporal_dist,
    aep_list=AEP_EVENTS,
    duration='24-hr',
    interval_min=6,
    output_dir='Design_Storms'  # Creates folder and saves CSVs
)

print("\n📊 STORM SUMMARY TABLE\n")
print(storm_summary.to_string(index=False))

### Comparative Visualization

Plot all storms together for comparison:

In [ ]:
# Create comparison plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Define colors for each storm
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(AEP_EVENTS)))

# Plot incremental precipitation
for i, aep in enumerate(AEP_EVENTS):
    storm = design_storms[aep]
    ax1.plot(storm['time_hr'], storm['incremental_in'], 
            label=f'{aep}-year', linewidth=2, color=colors[i], alpha=0.8)

ax1.set_ylabel('Incremental Precipitation (inches)', fontweight='bold', fontsize=11)
ax1.set_title(f'Design Storm Comparison - {PROJECT_NAME}\n24-Hour Duration | MSE 6 Distribution', 
             fontsize=13, fontweight='bold')
ax1.legend(loc='upper right', fontsize=10, framealpha=0.9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 24)

# Plot cumulative precipitation
for i, aep in enumerate(AEP_EVENTS):
    storm = design_storms[aep]
    ax2.plot(storm['time_hr'], storm['cumulative_in'], 
            label=f'{aep}-year: {storm["cumulative_in"].iloc[-1]:.2f} in', 
            linewidth=2.5, color=colors[i])

ax2.set_xlabel('Time (hours)', fontweight='bold', fontsize=11)
ax2.set_ylabel('Cumulative Precipitation (inches)', fontweight='bold', fontsize=11)
ax2.set_title('Cumulative Rainfall Curves', fontsize=13, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10, framealpha=0.9)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 24)

plt.tight_layout()
plt.savefig('Design_Storm_Comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comparison plot saved: Design_Storm_Comparison.png")

## Part 7: Professional Summary Tables 📋

### Create Report-Ready Tables

Generate tables suitable for engineering reports:

In [ ]:
# Enhanced summary table with additional statistics
def create_detailed_summary(storms_dict, atlas14_data, duration='24-hr'):
    """
    Create comprehensive summary table for engineering reports.
    """
    
    summary_rows = []
    
    for aep, storm in storms_dict.items():
        # Get Atlas 14 depth
        total_depth = atlas14_data.loc[duration, f'{aep}-yr']
        
        # Find peak
        peak_idx = storm['intensity_in_hr'].idxmax()
        
        # Calculate statistics
        row = {
            'Return Period': f'{aep}-year',
            'AEP (%)': f'{100/aep:.2f}',
            'Total Depth (in)': f'{total_depth:.2f}',
            'Peak Intensity (in/hr)': f'{storm.loc[peak_idx, "intensity_in_hr"]:.2f}',
            'Peak Time (hr)': f'{storm.loc[peak_idx, "time_hr"]:.1f}',
            'Duration (hr)': f'{storm["time_hr"].max():.1f}',
            'Time Steps': len(storm),
            'Interval (min)': int((storm['time_hr'].iloc[1] - storm['time_hr'].iloc[0]) * 60)
        }
        summary_rows.append(row)
    
    return pd.DataFrame(summary_rows)

# Create detailed summary
detailed_summary = create_detailed_summary(design_storms, atlas14_data)

print("\n" + "="*90)
print(f"DESIGN STORM SUMMARY - {PROJECT_NAME}")
print(f"Location: {PROJECT_LAT:.4f}°N, {PROJECT_LON:.4f}°W")
print(f"Temporal Distribution: MSE 6 24-Hour")
print(f"Date Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("="*90)
print("\n" + detailed_summary.to_string(index=False))
print("\n" + "="*90)

# Save to CSV
detailed_summary.to_csv('Design_Storm_Summary.csv', index=False)
print("\n✅ Summary table saved: Design_Storm_Summary.csv")

### Peak Intensity Comparison Table

In [ ]:
# Create peak intensity comparison
intensity_data = []

for aep, storm in design_storms.items():
    peak_idx = storm['intensity_in_hr'].idxmax()
    
    # Get peak 15-min, 1-hr, and 3-hr intensities
    peak_15min = storm.loc[peak_idx, 'incremental_in']
    peak_intensity = storm.loc[peak_idx, 'intensity_in_hr']
    
    # Calculate 1-hour total (10 consecutive 6-min intervals = 60 minutes)
    if peak_idx >= 5:
        one_hr_total = storm.loc[peak_idx-5:peak_idx+4, 'incremental_in'].sum()
    else:
        one_hr_total = storm.loc[:10, 'incremental_in'].sum()
    
    intensity_data.append({
        'Return Period': f'{aep}-year',
        'Peak 6-min (in)': f'{peak_15min:.3f}',
        'Peak Intensity (in/hr)': f'{peak_intensity:.2f}',
        'Max 1-hr Total (in)': f'{one_hr_total:.2f}'
    })

intensity_table = pd.DataFrame(intensity_data)

print("\n📊 PEAK INTENSITY COMPARISON\n")
print(intensity_table.to_string(index=False))

intensity_table.to_csv('Peak_Intensity_Summary.csv', index=False)
print("\n✅ Intensity table saved: Peak_Intensity_Summary.csv")

## Part 8: Export for H&H Models 💾

### HEC-HMS Format

Export in formats compatible with HEC-HMS:

In [ ]:
# Create HEC-HMS compatible exports
import zipfile

def export_for_hms(storms_dict, output_dir='HEC_HMS_Export'):
    """
    Export design storms in HEC-HMS compatible format.
    """
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    print(f"📤 Exporting for HEC-HMS...\n")
    
    for aep, storm in storms_dict.items():
        # Create HMS format (DateTime, Precipitation)
        hms_data = storm[['datetime', 'incremental_in']].copy()
        hms_data.columns = ['DateTime', 'Precipitation_in']
        
        # Save
        filename = output_path / f'Precip_{aep}yr_HMS.csv'
        hms_data.to_csv(filename, index=False)
        print(f"   ✅ {filename.name}")
    
    # Create README
    readme_path = output_path / 'README.txt'
    with open(readme_path, 'w') as f:
        f.write(f"HEC-HMS Precipitation Data\n")
        f.write(f"="*50 + "\n\n")
        f.write(f"Project: {PROJECT_NAME}\n")
        f.write(f"Location: {PROJECT_LAT:.4f}°N, {PROJECT_LON:.4f}°W\n")
        f.write(f"Source: NOAA Atlas 14\n")
        f.write(f"Temporal Distribution: MSE 6 24-Hour\n")
        f.write(f"Time Interval: 15 minutes\n")
        f.write(f"Duration: 24 hours\n\n")
        f.write(f"Files:\n")
        for aep in storms_dict.keys():
            f.write(f"  - Precip_{aep}yr_HMS.csv\n")
    
    print(f"\n✅ HEC-HMS export complete: {output_dir}/")

# Export all storms
export_for_hms(design_storms)

### Download All Files

Create a ZIP file with all outputs:

In [ ]:
# Create comprehensive ZIP file
import zipfile
from pathlib import Path

def create_project_archive(project_name):
    """
    Create ZIP archive with all project outputs.
    """
    zip_filename = f'{project_name.replace(" ", "_")}_Design_Storms.zip'
    
    print(f"📦 Creating project archive...\n")
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add design storm CSVs
        for file in Path('Design_Storms').glob('*.csv'):
            zipf.write(file, f'Design_Storms/{file.name}')
            print(f"   Added: {file.name}")
        
        # Add HEC-HMS exports
        for file in Path('HEC_HMS_Export').glob('*'):
            zipf.write(file, f'HEC_HMS_Export/{file.name}')
            print(f"   Added: {file.name}")
        
        # Add summary tables
        summary_files = ['Design_Storm_Summary.csv', 'Peak_Intensity_Summary.csv']
        for file in summary_files:
            if Path(file).exists():
                zipf.write(file, f'Summary_Tables/{file}')
                print(f"   Added: {file}")
        
        # Add comparison plot
        if Path('Design_Storm_Comparison.png').exists():
            zipf.write('Design_Storm_Comparison.png', 'Figures/Design_Storm_Comparison.png')
            print(f"   Added: Design_Storm_Comparison.png")
    
    print(f"\n✅ Archive created: {zip_filename}")
    return zip_filename

# Create archive
archive_file = create_project_archive(PROJECT_NAME)

# Download in Colab
print(f"\n📥 Downloading archive...")
files.download(archive_file)

## 🎉 What You Can Now Do!

Congratulations! You've completed Module 6 and mastered precipitation frequency analysis.

### ✅ You Can Now:

**Data Access:**
- Access NOAA Atlas 14 precipitation frequency data via API
- Handle connection errors and use manual data entry
- Understand precipitation frequency concepts (ARI, AEP)

**Design Storm Generation:**
- Upload and apply custom temporal distributions
- Generate design storm hyetographs at any time interval
- Create multiple AEP events in a single batch

**Professional Outputs:**
- Build comprehensive summary tables
- Create comparative visualizations
- Export data for HEC-HMS and HEC-RAS
- Package everything for project delivery

**Automation:**
- Batch process multiple return periods
- Generate consistent outputs across projects
- Create reproducible workflows

### 🚀 Real-World Applications:

You can now:
- Generate complete design storm suites for flood studies
- Support FEMA floodplain analyses
- Prepare precipitation inputs for H&H models
- Create professional engineering reports
- Automate repetitive precipitation analysis tasks

### 📚 Key Functions You've Learned:

```python
# Get Atlas 14 data
atlas14_data = get_atlas14_data(lat, lon)

# Generate single storm
storm = generate_design_storm(total_depth, temporal_dist)

# Batch process multiple AEPs
storms, summary = generate_aep_suite(atlas14_data, temporal_dist, [10, 50, 100])

# Export for HMS
export_for_hms(storms)
```

### 💡 Best Practices:

1. **Always document** your Atlas 14 source and date
2. **Verify** temporal distribution is appropriate for your region
3. **Check** time intervals match model requirements
4. **Create** summary tables for every project
5. **Archive** all inputs and outputs together

### 🔗 Next Steps:

- Apply to your own projects
- Customize temporal distributions
- Integrate with HEC-HMS/RAS workflows
- Build project-specific templates

**You're now ready to handle precipitation frequency analysis professionally!** ☔🎓

## Appendix: Quick Reference 📋

### Common AEP/ARI Conversions

| AEP | ARI | Use Case |
|-----|-----|----------|
| 50% | 2-yr | Frequent flooding |
| 20% | 5-yr | Nuisance flooding |
| 10% | 10-yr | Minor flooding, drainage design |
| 4% | 25-yr | Moderate flooding |
| 2% | 50-yr | Major flooding |
| 1% | 100-yr | **Base flood elevation (FEMA)** |
| 0.5% | 200-yr | Freeboard |
| 0.2% | 500-yr | Critical infrastructure |

### Quick Workflow

```python
# 1. Get Atlas 14 data
atlas14_data = get_atlas14_data(lat, lon)

# 2. Upload temporal distribution
temporal_dist = pd.read_csv('distribution.csv')

# 3. Generate storms
storms, summary = generate_aep_suite(
    atlas14_data, temporal_dist, [10, 50, 100]
)

# 4. Export
export_for_hms(storms)
```

### Typical Time Intervals

- **6 minutes (0.1 hr)**: High-resolution urban hydrology, detailed storm analysis
- **15 minutes**: Standard urban hydrology
- **1 hour**: Most common for H&H modeling
- **6 hours**: Large watersheds, simplified analysis
- **1 day**: Long-duration events

### File Outputs

```
Project/
├── Design_Storms/
│   ├── Storm_10yr_24-hr.csv
│   ├── Storm_50yr_24-hr.csv
│   └── Storm_100yr_24-hr.csv
├── HEC_HMS_Export/
│   ├── Precip_10yr_HMS.csv
│   └── README.txt
├── Summary_Tables/
│   ├── Design_Storm_Summary.csv
│   └── Peak_Intensity_Summary.csv
└── Figures/
    └── Design_Storm_Comparison.png
```

Keep this handy for quick reference!